In [1]:
import torch
from torch import nn
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
from d2l import torch as d2l

In [8]:
def corr2d_multi_in(X, K):
    return sum(d2l.corr2d(x, k) for x, k in zip(X, K))

In [34]:
X = torch.tensor([[[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]],
               [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]]])
K = torch.tensor([[[0.0, 1.0], [2.0, 3.0]], [[1.0, 2.0], [3.0, 4.0]]])
print(K.shape)

corr2d_multi_in(X, K)

torch.Size([2, 2, 2])


tensor([[ 56.,  72.],
        [104., 120.]])

In [39]:
def corr2d_multi_in_out(X, K):
    # 迭代“K”的第0个维度，每次都对输入“X”执行互相关运算。
    # 最后将所有结果都叠加在一起
    return torch.stack([corr2d_multi_in(X, k) for k in K], 0)

In [35]:
K = torch.stack((K, K + 1, K + 2), 0)
K.shape

torch.Size([3, 2, 2, 2])

In [40]:
corr2d_multi_in_out(X, K)

tensor([[[ 56.,  72.],
         [104., 120.]],

        [[ 76., 100.],
         [148., 172.]],

        [[ 96., 128.],
         [192., 224.]]])

In [46]:
def corr2d_multi_in_out_1x1(X, K):
    # 唯一运算发生在通道，可视作将c_i个输入转换为c_o个输出的全连接层
    c_i, h, w = X.shape
    c_o = K.shape[0]
    X = X.reshape((c_i, h * w)) # 丢弃空间信息
    K = K.reshape((c_o, c_i)) # 由于为 1 x 1卷积核，故后两维度 (1 x 1) 可直接丢弃
    Y = torch.matmul(K, X)
    return Y.reshape((c_o, h, w))

In [48]:
X = torch.normal(0, 1, (3, 3, 3))
K = torch.normal(0, 1, (2, 3, 1, 1))

Y1 = corr2d_multi_in_out_1x1(X, K)
Y2 = corr2d_multi_in_out(X, K)
Y1, Y2
# assert float(torch.abs(Y1 - Y2).sum()) < 1e-6

(tensor([[[-1.4057, -1.3876,  0.6373],
          [-0.1617,  1.5570, -0.2042],
          [-0.7726, -0.7967,  0.5489]],
 
         [[ 0.7820,  0.2284,  0.5667],
          [-0.1662, -1.1327,  0.1110],
          [ 0.4458,  1.0283,  0.0151]]]),
 tensor([[[-1.4057, -1.3876,  0.6373],
          [-0.1617,  1.5570, -0.2042],
          [-0.7726, -0.7967,  0.5489]],
 
         [[ 0.7820,  0.2284,  0.5667],
          [-0.1662, -1.1327,  0.1110],
          [ 0.4458,  1.0283,  0.0151]]]))